# PyVISTA 3D Visualization of seismic data

In the notebook we use PyVista to create 3D plots of the seismic data used for the time benchmarking of ChirpRadon3D and PhaseShift3D operators.

**Note**: .png files can only be creating running this as a Python script or in a IPython console.

In [1]:
import pyvista as pv

from pylops.utils.seismicevents import *
from pylops.utils.wavelets import *
from pylops.utils.tapers import *
from pylops.basicoperators import *
from pylops.signalprocessing import *
from pylops.waveeqprocessing import *

pv.set_plot_theme("document")
print(pv.Report())


--------------------------------------------------------------------------------
  Date: Sat Jul 24 11:54:58 2021 +03

                OS : Darwin
            CPU(s) : 16
           Machine : x86_64
      Architecture : 64bit
       Environment : Jupyter
        GPU Vendor : ATI Technologies Inc.
      GPU Renderer : AMD Radeon Pro 5500M OpenGL Engine
       GPU Version : 4.1 ATI-3.10.19

  Python 3.8.8 (default, Apr 13 2021, 12:59:45)  [Clang 10.0.0 ]

           pyvista : 0.31.3
               vtk : 8.2.0
             numpy : 1.19.2
           imageio : 2.9.0
           appdirs : 1.4.4
            scooby : 0.5.7
            meshio : 4.4.6
        matplotlib : 3.3.4
             PyQt5 : 5.9.2
           IPython : 7.22.0
        ipyvtklink : 0.2.1
             scipy : 1.6.2
              tqdm : 4.61.2

  Intel(R) Math Kernel Library Version 2019.0.4 Product Build 20190411 for
  Intel(R) 64 architecture applications
----------------------------------------------------------------------

## ChirpRadon3D

In [2]:
par = {'ot': 0,    'dt': 0.004, 'nt': 201,
       'ox': -625, 'dx': 12.5, 'nx': 101,
       'oy': -625, 'dy': 12.5, 'ny': 101,
       'f0': 20}
theta = [30, ]
phi = [0, ]
t0 = [0.5, ]
amp = [1., ]

# Create axis
t, t2, x, y = makeaxis(par)
dt, dx, dy = par['dt'], par['dx'], par['dy']

# Create wavelet
wav = ricker(t[:41], f0=par['f0'])[0]

# Generate model
_, d = linear3d(x, y, t, 1500., t0, theta, phi, amp, wav)

npy, pymax = par['ny'], 5e-4
npx, pxmax = par['nx'], 5e-4

py = np.linspace(-pymax, pymax, npy)
px = np.linspace(-pxmax, pxmax, npx)
dpy = np.abs(py[1]-py[0])
dpx = np.abs(px[1]-px[0])

R3Op = ChirpRadon3D(t, y, x, (pymax*dy/dt, pxmax*dx/dt), dtype='float64')

dL_chirp = R3Op * d.ravel()
dadj_chirp = R3Op.H * dL_chirp
dinv_chirp = R3Op.inverse(dL_chirp)

dL_chirp = dL_chirp.reshape(par['ny'], par['nx'], par['nt'])
dadj_chirp = dadj_chirp.reshape(par['ny'], par['nx'], par['nt'])
dinv_chirp = dinv_chirp.reshape(par['ny'], par['nx'], par['nt'])

In [3]:
grid = pv.UniformGrid()
grid.dimensions = np.array(d.shape) + 1
grid.origin = (0, 0, 0)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_arrays["values"] = np.asfortranarray(d).flatten(order="F")#d.flatten(order="F")  # Flatten the array!

slices = grid.slice_orthogonal(z=int(t0[0]/par['dt']))

#slices.plot(notebook=True, cmap='seismic', clim=[-1,1], 
#            #background='white', 
#            opacity=1, lighting=True)

plotter = pv.Plotter(off_screen=True)
plotter.add_mesh(slices, cmap='seismic', clim=[-1,1], 
                 opacity=1, lighting=True)
plotter.show(screenshot='Data.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [4]:
grid = pv.UniformGrid()
grid.dimensions = np.array(dL_chirp.shape) + 1
grid.origin = (0, 0, 0)  # The botom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_arrays["values"] = np.asfortranarray(dL_chirp).flatten(order="F")#d.flatten(order="F")  # Flatten the array!

slices = grid.slice_orthogonal(z=int(t0[0]/par['dt']))
plotter = pv.Plotter(off_screen=True)
plotter.add_mesh(slices, cmap='seismic', clim=[-1e4,1e4], 
                 opacity=1, lighting=True)
plotter.show(screenshot='Radon.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## PhaseShift

In [5]:
par = {'ox':-1000, 'dx':20, 'nx':101,
       'oy':-1000, 'dy':20, 'ny':101,
       'ot':0, 'dt':0.004, 'nt':151,
       'f0': 20, 'nfmax': 210}

# Create axis
t, t2, x, y = makeaxis(par)

# Create wavelet
wav = ricker(np.arange(41) * par['dt'], f0=par['f0'])[0]

vrms = [2000, 2200, 3000]
t0 = [0.1, 0.2, 0.25]
amp = [1., 0.6, -2.]

_, m = hyperbolic3d(x, y, t, t0, vrms, vrms, amp, wav)

pad = 11
taper = taper3d(par['nt'], (par['ny'], par['nx']), (3, 3))
mpad = np.pad(m*taper, ((pad, pad), (pad, pad), (0, 0)), mode='constant')

vel = 1500.
zprop = 200
freq = np.fft.rfftfreq(par['nt'], par['dt'])
kx = np.fft.fftshift(np.fft.fftfreq(par['nx'] + 2*pad, par['dx']))
ky = np.fft.fftshift(np.fft.fftfreq(par['ny'] + 2*pad, par['dy']))
Pop = PhaseShift(vel, zprop, par['nt'], freq, kx, ky)

mdown = Pop * mpad.transpose(2, 1, 0).ravel()

mup = Pop.H * mdown.ravel()

mdown = np.real(mdown.reshape(par['nt'],
                              par['nx'] + 2 * pad,
                              par['ny'] + 2 * pad)[:, pad:-pad, pad:-pad])
mup = np.real(mup.reshape(par['nt'],
                          par['nx'] + 2 * pad,
                          par['ny'] + 2 * pad)[:, pad:-pad, pad:-pad])
mdown = mdown.transpose(1, 2, 0)
mup = mup.transpose(1, 2, 0)

In [6]:
grid = pv.UniformGrid()
grid.dimensions = np.array(m.shape) + 1
grid.origin = (0, 0, 0)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_arrays["values"] = np.flip(m, axis=-1).flatten(order="F")  # Flatten the array!

slices = grid.slice_orthogonal()
plotter = pv.Plotter(off_screen=True)
plotter.add_mesh(slices, cmap='seismic', clim=[-2,2], 
                 opacity=1, lighting=True)
plotter.show(screenshot='PhaseShift_Data.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [ ]:
# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape + 1 because we want to inject our values on
#   the CELL data
grid.dimensions = np.array(mdown.shape) + 1

# Edit the spatial reference
grid.origin = (0, 0, 0)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis

# Add the data values to the cell data
grid.cell_arrays["values"] = np.flip(mdown, axis=-1).flatten(order="F")  # Flatten the array!

slices = grid.slice_orthogonal()

#slices.plot(notebook=True, cmap='seismic', clim=[-1,1], 
#            #background='white', 
#            opacity=1, lighting=True)

plotter = pv.Plotter(off_screen=True)
plotter.add_mesh(slices, cmap='seismic', clim=[-2,2], 
                 opacity=1, lighting=True)
plotter.show(screenshot='PhaseShift_Forward.png')

In [ ]:
# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape + 1 because we want to inject our values on
#   the CELL data
grid.dimensions = np.array(mup.shape) + 1

# Edit the spatial reference
grid.origin = (0, 0, 0)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis

# Add the data values to the cell data
grid.cell_arrays["values"] = np.flip(mup, axis=-1).flatten(order="F")  # Flatten the array!

slices = grid.slice_orthogonal()

#slices.plot(notebook=True, cmap='seismic', clim=[-1,1], 
#            #background='white', 
#            opacity=1, lighting=True)

plotter = pv.Plotter(off_screen=True)
plotter.add_mesh(slices, cmap='seismic', clim=[-2,2], 
                 opacity=1, lighting=True)
plotter.show(screenshot='PhaseShift_Backward.png')